In [ ]:
import os 
import subprocess
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from unstructured.partition.docx import partition_docx
from unstructured.partition.pdf import partition_pdf
from IPython.display import HTML

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
# %pip install LibreOffice

In [ ]:
filepath="pdfs/Documentation-Project.docx"
output_dir = "pdfs"

In [ ]:
loader = UnstructuredWordDocumentLoader(
    file_path=filepath,
    mode="elements",
    strategy="hi_res"
)

docs=loader.load()

In [ ]:
def extract_docx_elements(filepath: str,output_dir:str) -> list:
    elements = partition_docx(
        filename=filepath,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=['Image'],
        hi_res_model_name="yolox",
        extract_image_block_to_payload=True,
        extract_image_block_output_dir=output_dir
    )
    print("Total elements extracted -> ",len(elements))
    return elements

elements= extract_docx_elements(filepath,output_dir)

In [ ]:
elements

In [ ]:
%pip install docx2txt

In [ ]:
import docx2txt

In [ ]:
text = docx2txt.process(filepath,output_dir)

In [ ]:
table=[]
images=[]

In [ ]:
elements[76].to_dict()

In [ ]:
set([str(type(el))for el in elements])

In [ ]:
def display_table(tables):
    for i,table_obj in enumerate(tables):
        display(HTML(table_obj))

display_table(table)

In [ ]:
%pip install docx2pdf

In [ ]:
import subprocess, os

SOFFICE_PATH = r"C:\Program Files\LibreOffice\program\soffice.exe"

def docx_to_pdf(docx_path, out_dir="pdfs"):
    os.makedirs(out_dir, exist_ok=True)

    subprocess.run([
        SOFFICE_PATH,
        "--headless",
        "--convert-to", "pdf",
        docx_path,
        "--outdir", out_dir
    ], check=True)

    return os.path.join(
        out_dir,
        os.path.basename(docx_path).replace(".docx", ".pdf")
    )


pdf_dir = docx_to_pdf(filepath,output_dir)

In [ ]:
def extract_pdf_elements(filepath: str,output_dir:str) -> list:
    elements = partition_pdf(
        filename=filepath,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=['Image'],
        extract_image_block_to_payload=True,
        extract_image_block_output_dir=output_dir
    )
    print("Total elements extracted -> ",len(elements))
    return elements

elements = extract_pdf_elements(pdf_dir,output_dir)

In [ ]:
set([str(type(el)) for el in elements])

In [ ]:
tables = [elem for elem in elements if elem.category == "Table"]
images = [elem for elem in elements if elem.category == "Image"]

In [ ]:
import base64

In [ ]:
output_dir="docx_images"

In [ ]:
def get_image(output_dir, image_list):
    os.makedirs(output_dir, exist_ok=True)
    for i, img_obj in enumerate(image_list):
        bs64_str = img_obj.to_dict()['metadata']['image_base64']
        image_data = base64.b64decode(bs64_str)

        path = os.path.join(output_dir, f"image-{i}.png")

        with open(path, "wb") as f:
            f.write(image_data)

get_image(output_dir,images)